In [1]:
from sklearn.datasets import fetch_openml
import matplotlib as mpl
import matplotlib.pyplot as plt
import random as rd
import numpy as np
mnist = fetch_openml("mnist_784",version=1)
mnist.keys()

np.save("mnist_data.npy", mnist.data)
np.save("mnist_target.npy", mnist.target)

#30s

c:\Users\MaxB2\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
"""Value type tranformation and train-test split"""
X,y = mnist["data"],mnist["target"]
print("Before: ",type(y[rd.randint(0,X.shape[0]-1)]))
y = y.astype(np.uint8)
print("After: ",type(y[rd.randint(0,X.shape[0]-1)]))

X_train, X_test, y_train, y_test = X[:60000],X[60000:],y[:60000],y[60000:]  

Before:  <class 'str'>
After:  <class 'numpy.uint8'>


In [3]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()

from sklearn.model_selection import GridSearchCV as GS

param_grid = [
    {"weights":['uniform', 'distance'],"n_neighbors":[1,2,4,8]},
]

grid_search = GS(KNN,
                 param_grid,
                 cv=5,
                 scoring="neg_mean_squared_error",
                 return_train_score=True
)

"""The Best Params"""
grid_search.fit(X_train,y_train) # Takes a while
best_parameters = grid_search.best_params_
print("Best Paramaters: ", best_parameters)
best_estimator = grid_search.best_estimator_
print("Best Estimators: ",best_estimator)

"""Printing Combinations And Respective Results"""
print("Combinations And Respective Results")
cvres = grid_search.cv_results_
for mean_score, params in zip (cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),params)

KeyboardInterrupt: 

In [60]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

best_estimator.fit(X_train,y_train)
accuracy_scores = cross_val_score(best_estimator,X_train,y_train,cv=3, scoring="accuracy")
best_predictions = best_estimator.predict(X_train)
np.mean(accuracy_scores) #0.9716166666666666



In [ ]:
"""Number Demonstration"""

index = rd.randint(0,X.shape[0]-1)


def plot_digit(Data,Index):
    digit = np.array(Data.iloc[Index])
    digit1 = digit.reshape(28,28)
    plt.imshow(digit1,cmap="binary")
    plt.axis("off")
    plt.show()

"""Image Shifter"""


def image_rearrange(Data,Index):
    digit = np.array(Data.iloc[Index])
    digit1 = digit.reshape(28,28)
    



print("Index: ", index,"  Label: ",y[index])
plot_digit(X,index)

In [ ]:
"""5-Only Identificiation"""

y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

from sklearn.linear_model import SGDClassifier
SGD = SGDClassifier(random_state=69)
SGD.fit(X_train, y_train_5)

from sklearn.model_selection import cross_val_score
cross_val_score(SGD, X_train, y_train_5, cv=3, scoring="accuracy")

digit_0 = np.array(X.iloc[0])
temp_cnt = 0
tests_it = 120
while temp_cnt <= tests_it:
    temp_index = rd.randint(0,X_train.shape[0]-1)
    print("Index: ", temp_index, "Target: ", y_train[temp_index])
    print(SGD.predict([np.array(X.iloc[temp_index])])) # YAY
    temp_cnt += 1
else:
    tests_it - temp_cnt
    pass

from sklearn.model_selection import cross_val_predict
y_train5_pred = cross_val_predict(SGD, X_train, y_train_5, cv=3)
print(y_train5_pred)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train_5,y_train5_pred))


from sklearn.metrics import precision_score, recall_score, f1_score
print("Precision: ",precision_score(y_train_5,y_train5_pred)) # Percentage of pictures correctly identified as 5
print("Recall/Sensitivity: ",recall_score(y_train_5,y_train5_pred)) # Percentage of identified 5s of all 5s
print("F1 Score :",f1_score(y_train_5,y_train5_pred)) # Harmonic mean of Sensitivity and precision

y_scores = cross_val_predict(SGD, X_train, y_train_5, cv=3, method="decision_function")
print(y_scores)

"""
PR Curve (Precision Recall Curve)
Use case: Positive category is rare, False positives more important than false negatives
Otherwise: ROC Curve
"""
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)
def plot_precision_recall_vs_threshold(precisions,recalls,thresholds):
    plt.plot(thresholds,precisions[:-1],"b--",label="Precision")
    plt.plot(thresholds,recalls[:-1],"g-",label="Sensitivity")

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()
plt.plot(recalls[:-1], precisions[:-1])
plt.title("Precision/Sensitivity Tradeoff Visualized")
plt.xlabel("Recall/Sensitivity")
plt.ylabel("Precision")
plt.show()

"""
ROC Curve (Receiver Operating Characteristic)
True Positive Rate = Sensitivity
False Positive Rate = Specificity
"""

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)
def plot_roc_curve(fpr,tpr,label=None):
    plt.plot(fpr,tpr,linewidth=2,label=label)
    plt.plot([0,1],[0,1],"k--")
    plt.title("Specificity/Sensitivity Tradeoff Visualized")
    plt.xlabel("Specificity")
    plt.ylabel("Sensitivity")
plot_roc_curve(fpr,tpr)
plt.show()

from sklearn.metrics import roc_auc_score
print("ROC AUC Score: ",roc_auc_score(y_train_5, y_scores))

# SEE Applications for Sensitivity vs Relevance Modification

threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
threshold_90_precision

y_train_pred_90 = (y_scores > threshold_90_precision)
print("Precision: ", precision_score(y_train_5, y_train_pred_90))
print("Precision: ", recall_score(y_train_5, y_train_pred_90))


In [ ]:
"""Various Testing With BS Models"""

from sklearn.svm import SVC
SVM = SVC()
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_train[:10])
y_pred
#2m43s

svm_scores = SVM.decision_function(X_train[:10])
svm_scores1 = SVM.decision_function(X_train[:1])
np.argmax(svm_scores1)

from sklearn.multiclass import OneVsRestClassifier

OVRC = OneVsRestClassifier(SVC())
#OVRC.fit(X_train,y_train)
# #OVRC.predict(X_train[:10])

# takes 4ever

from sklearn.multiclass import OneVsOneClassifier
OVOC = OneVsOneClassifier(SVC())
#OVOC.fit(X_train,y_train)
#y_pred = OVOC.predict(X_train[:10])
# takes 4 ever ever ever

SGD.fit(X_train,y_train)
y_pred = SGD.predict(X_train[:10])
print(y_pred)
print(SGD.decision_function(X_train[:10]))
#1m47

from sklearn.ensemble import RandomForestRegressor
RFR = RandomForestRegressor()


# RFR.fit(X_train,y_train) # fit takes forever
# y_pred = RFR.predict(X_train[:10])

#print(y_pred)
#print(RFR.decision_path(X_train[:10]))

cross_val_score(SGD,X_train,y_train,cv=5, scoring="accuracy")
#7m

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))

cross_val_score(SGD,X_train_scaled,y_train,cv=5, scoring="accuracy")
#18m

"""Train Predictions"""
#y_pred_sgd = cross_val_predict(SGD,X_train_scaled, y_train, cv=5)

"""Error Analysis"""

#cm = confusion_matrix(y_train, y_pred_sgd)
#row_sums = cm.sum(axis=1,keepdims=True)
#norm_conf_mx = cm / row_sums
#np.fill_diagonal(norm_conf_mx,0)
#plt.matshow(norm_conf_mx,cmap=plt.cm.gray)
#plt.show()

In [ ]:
"""KNN First Tries"""

y_multilabel = np.c_[(y_train >= 7),(y_train % 2 == 1)]
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()

KNN.fit(X_train, y_multilabel)
y_predict_multilabel = KNN.predict(X_train)
F1_a = f1_score(y_multilabel, y_predict_multilabel, average="macro")
print("F1 Unscaled + No CV: ",F1_a)

KNN.fit(X_train_scaled, y_multilabel)
y_predict_multilabel = KNN.predict(X_train_scaled)
F1_b = f1_score(y_multilabel, y_predict_multilabel, average="macro")
print("F1 Scaled + No CV:",F1_b)

y_predict_cv_multilabel = cross_val_predict(KNN, X_train_scaled, y_multilabel,cv=3)
F1_c = f1_score(y_multilabel, y_predict_cv_multilabel, average="macro")
print("F1 Scaled + CV: ", F1_c)

KNN.fit(X_train, y_multilabel)
y_predict_multilabel = cross_val_predict(KNN, X_train, y_multilabel,cv=3)
F1_d = f1_score(y_multilabel, y_predict_multilabel, average="macro")

print("F1 Unscaled + CV: ", F1_d)

In [ ]:
noise = np.random.randint(0,100,size=(len(X_train),784))
X_train_noise = X_train + noise
noise = np.random.randint(0,100,size=(len(X_test),784))
X_test_noise = X_test + noise

y_train_clean = np.copy(X_train)
y_test_clean = np.copy(X_test)

KNN.fit(X_train_noise,y_train_clean) # takes 4 days
cleaned_digit_reg = KNN.predict(X_train_noise)


"""Noised Number Demonstration"""
index1 = rd.randint(0,X_train_noise.shape[0]-1)
print("Index: ", index1)



some_digit_noisy = np.array(X_train_noise.iloc[index1])
#print(X_train_noise.iloc[index1])
some_digit_noisy_image = some_digit_noisy.reshape(28,28)
plt.imshow(some_digit_noisy_image,cmap="binary")
plt.axis("off")
plt.show()

some_digit_clean = np.array(y_train_clean[index1])
#print(y_train_clean[index1])
some_digit_clean_image = some_digit_clean.reshape(28,28)
plt.imshow(some_digit_clean_image,cmap="binary")
plt.axis("off")
plt.show()